## Data extraction from server

In [1]:
# lets extract the data from the server as individual device json files
# when we began data collection, it was in trial phase and we used 1 device - "smriti phone".
# once the collection was successful, we renamed the 3 devices to be device 1, device 3 and device 4

import json

In [5]:
# importing device 1 data as json

!curl http://143.198.161.82:56789/get/device1 --output device1.json

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  142M    0  142M    0     0  1585k      0 --:--:--  0:01:31 --:--:-- 2407k-:--  0:00:10 --:--:-- 1089k--:--  0:00:50 --:--:-- 1467k109M    0     0  1504k      0 --:--:--  0:01:14 --:--:-- 2344k--:--:--  0:01:27 --:--:-- 1681k


In [4]:
# importing device 3 data as json

!curl http://143.198.161.82:56789/get/device3 --output device3.json

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 6443k    0 6443k    0     0  1554k      0 --:--:--  0:00:04 --:--:-- 1554k


In [31]:
# importing device 4 data as json

!curl http://143.198.161.82:56789/get/device4 --output device4.json

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 5726k    0 5726k    0     0  1269k      0 --:--:--  0:00:04 --:--:-- 1542k


In [3]:
# importing smritiphone data as json

!curl http://143.198.161.82:56789/get/smriti%20phone --output smritiphone.json

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 3650k    0 3650k    0     0  1237k      0 --:--:--  0:00:02 --:--:-- 1237k


## Device1

In [ ]:
# for each device, data needs to be merged with the gps locations for drawing insights on location of beacons and
# beacon signals. We will read the data from each json and do the necessary data cleaning before merging with 
# gps data on the timestamps. 

In [8]:
import urllib.request, json
import requests
import numpy as np
import pandas as pd
from itertools import chain

In [9]:
records = []

file_path = '/Users/smritimohta/Desktop/Capstone/Final/Code/device1.json'
with open(file_path) as f:
    for line in f:
        records.append(json.loads(line))
main = pd.DataFrame.from_dict(records)

In [10]:
beacons = main["beacons"]
beacondetails = pd.DataFrame(list(chain.from_iterable(beacons)))
beacondata = beacondetails["ibeacon_data"]
beacondatadf = pd.DataFrame(beacondata)
beacondatadetails = beacondatadf["ibeacon_data"].apply(pd.Series)
detailsdf = pd.concat([main, beacondetails, beacondatadetails],axis=1)
cleandf = detailsdf[['timestamp','name','beacon_type','distance','hashcode','isBlocked','last_seen','manufacturer','rssi','tx_power','eddystoneUidData','major','minor','uuid']]
finaldf = cleandf[cleandf["timestamp"].notnull()]
finaldf['timestamp']=pd.to_datetime(finaldf['timestamp'])

<ipython-input-10-2a1b208b628d>:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  finaldf['timestamp']=pd.to_datetime(finaldf['timestamp'])


In [4]:
finaldf

,timestamp,name,beacon_type,distance,hashcode,isBlocked,last_seen,manufacturer,rssi,tx_power,eddystoneUidData,major,minor,uuid
0,2021-09-22 22:57:06,device1,ibeacon,6.826854,-1778984009,False,1632351424960,76,-90,-61,NaN,20089,47339,5993a94c-7d97-4df7-9abf-e493bfd5d000
1,2021-09-22 22:57:08,device1,ibeacon,16.734379,-1060113490,False,1632351424958,76,-91,-55,NaN,1,0,2686f39c-bada-4658-854a-a62e7e5e8b8d
2,2021-09-22 22:57:10,device1,ibeacon,6.826854,32202963,False,1632351426135,76,-90,-61,NaN,13834,55446,5993a94c-7d97-4df7-9abf-e493bfd5d000
3,2021-09-22 22:57:12,device1,ibeacon,10.732786,-1060113490,False,1632351428336,76,-86,-55,NaN,1,0,2686f39c-bada-4658-854a-a62e7e5e8b8d
4,2021-09-22 22:57:15,device1,ibeacon,5.919498,-1229677222,False,1632351430543,76,-88,-61,NaN,43769,41619,5993a94c-7d97-4df7-9abf-e493bfd5d000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
331195,2021-10-23 00:06:44,device1,eddystone_uid,4.969997,1015808970,False,1634936683029,65194,-80,-60,"{'instance_id': '0x3f0070250000', 'namespace_i...",NaN,NaN,NaN
331196,2021-10-23 00:06:44,device1,eddystone_uid,4.156745,1093042504,False,1634936683120,65194,-75,-60,"{'instance_id': '0x3f0091240000', 'namespace_i...",NaN,NaN,NaN
331197,2021-10-23 00:06:44,device1,eddystone_uid,4.121900,1190557106,False,1634936682995,65194,-85,-60,"{'instance_id': '0x3f0067250000', 'namespace_i...",NaN,NaN,NaN
331198,2021-10-23 00:06:44,device1,eddystone_uid,4.802511,1198475487,False,1634936683290,65194,-87,-60,"{'instance_id': '0x3f00a0240000', 'namespace_i...",NaN,NaN,NaN


In [12]:
filenames = ['20210924-151809','20211008-163603','20211012-132348',\
             '20211013-143524','20211015-201017','20211016-175814','20211018-203305','20211021-185114',\
            '20211022-162321']

gps_device1 = []

for file in filenames:
    GPS = pd.read_csv('/Users/smritimohta/Desktop/Capstone/Final/Code/GPS Logger Collection/Device1/device1 - '+file+'.txt',header=0)
    gps_device1.append(GPS)
    
gps_df = pd.concat(gps_device1, axis=0, ignore_index=True)

In [13]:
gps_df

,type,date time,latitude,longitude,accuracy(m),altitude(m),geoid_height(m),speed(m/s),bearing(deg),sat_used,sat_inview,name,desc
0,T,2021-09-24 19:18:09,40.702276,-74.012627,14,-25.684,NaN,0.31,141.0,21.0,31,20210924-151809,GPS Logger: 20210924-151809
1,T,2021-09-24 19:18:10,40.702266,-74.012618,13,-24.498,NaN,0.00,NaN,21.0,31,NaN,NaN
2,T,2021-09-24 19:18:11,40.702276,-74.012578,11,-29.481,NaN,0.00,NaN,21.0,31,NaN,NaN
3,T,2021-09-24 19:18:12,40.702286,-74.012567,10,-28.212,NaN,0.00,NaN,22.0,31,NaN,NaN
4,T,2021-09-24 19:18:13,40.702283,-74.012560,9,-27.654,NaN,0.00,NaN,22.0,31,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
56025,T,2021-10-23 00:06:44,40.749660,-73.987924,6,0.729,NaN,1.00,346.0,18.0,25,NaN,NaN
56026,T,2021-10-23 00:06:45,40.749666,-73.987926,7,2.142,NaN,0.44,345.0,18.0,25,NaN,NaN
56027,T,2021-10-23 00:06:46,40.749674,-73.987924,7,3.031,NaN,0.72,346.0,18.0,25,NaN,NaN
56028,T,2021-10-23 00:06:47,40.749679,-73.987923,7,2.002,NaN,0.44,346.0,18.0,25,NaN,NaN


In [14]:
gps_df['date time']= pd.to_datetime(gps_df['date time'])

In [15]:
df_device1 = finaldf.merge(gps_df, left_on='timestamp', right_on='date time')
df_device1

,timestamp,name_x,beacon_type,distance,hashcode,isBlocked,last_seen,manufacturer,rssi,tx_power,...,longitude,accuracy(m),altitude(m),geoid_height(m),speed(m/s),bearing(deg),sat_used,sat_inview,name_y,desc
0,2021-09-24 19:18:47,device1,ibeacon,4.600171,-1229677222,False,1632351432738,76,-81,-61,...,-74.012588,5,-33.543,NaN,0.87,360.0,20.0,31,NaN,NaN
1,2021-09-24 19:18:48,device1,ibeacon,6.358023,32202963,False,1632351432737,76,-88,-61,...,-74.012594,5,-33.221,NaN,0.73,359.0,20.0,31,NaN,NaN
2,2021-09-24 19:19:16,device1,ibeacon,4.040893,144680941,False,1632511100375,76,-80,-59,...,-74.012693,6,-32.870,NaN,0.00,NaN,21.0,31,NaN,NaN
3,2021-09-24 19:19:23,device1,ibeacon,8.462784,144680941,False,1632511125686,76,-90,-59,...,-74.012701,6,-32.694,NaN,0.00,NaN,21.0,31,NaN,NaN
4,2021-09-24 19:19:25,device1,ibeacon,7.056213,144680941,False,1632511127891,76,-85,-59,...,-74.012702,6,-32.653,NaN,0.00,NaN,21.0,31,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
326981,2021-10-23 00:06:44,device1,eddystone_uid,4.969997,1015808970,False,1634936683029,65194,-80,-60,...,-73.987924,6,0.729,NaN,1.00,346.0,18.0,25,NaN,NaN
326982,2021-10-23 00:06:44,device1,eddystone_uid,4.156745,1093042504,False,1634936683120,65194,-75,-60,...,-73.987924,6,0.729,NaN,1.00,346.0,18.0,25,NaN,NaN
326983,2021-10-23 00:06:44,device1,eddystone_uid,4.121900,1190557106,False,1634936682995,65194,-85,-60,...,-73.987924,6,0.729,NaN,1.00,346.0,18.0,25,NaN,NaN
326984,2021-10-23 00:06:44,device1,eddystone_uid,4.802511,1198475487,False,1634936683290,65194,-87,-60,...,-73.987924,6,0.729,NaN,1.00,346.0,18.0,25,NaN,NaN


In [17]:
df_device1.columns

Index(['timestamp', 'name_x', 'beacon_type', 'distance', 'hashcode',
       'isBlocked', 'last_seen', 'manufacturer', 'rssi', 'tx_power',
       'eddystoneUidData', 'major', 'minor', 'uuid', 'type', 'date time',
       'latitude', 'longitude', 'accuracy(m)', 'altitude(m)',
       'geoid_height(m)', 'speed(m/s)', 'bearing(deg)', 'sat_used',
       'sat_inview', 'name_y', 'desc'],
      dtype='object')

## device 3

In [18]:
records = []

file_path = '/Users/smritimohta/Desktop/Capstone/Final/Code/device3.json'
with open(file_path) as f:
    for line in f:
        records.append(json.loads(line))
main = pd.DataFrame.from_dict(records)
beacons = main["beacons"]
beacondetails = pd.DataFrame(list(chain.from_iterable(beacons)))
beacondata = beacondetails["ibeacon_data"]
beacondatadf = pd.DataFrame(beacondata)
beacondatadetails = beacondatadf["ibeacon_data"].apply(pd.Series)
detailsdf = pd.concat([main, beacondetails, beacondatadetails],axis=1)
cleandf = detailsdf[['timestamp','name','beacon_type','distance','hashcode','isBlocked','last_seen','manufacturer','rssi','tx_power','eddystoneUidData','major','minor','uuid']]
finaldf = cleandf[cleandf["timestamp"].notnull()]
finaldf['timestamp']=pd.to_datetime(finaldf['timestamp'])

<ipython-input-18-aaabada1439b>:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  finaldf['timestamp']=pd.to_datetime(finaldf['timestamp'])


In [19]:
finaldf

,timestamp,name,beacon_type,distance,hashcode,isBlocked,last_seen,manufacturer,rssi,tx_power,eddystoneUidData,major,minor,uuid
0,2021-09-17 18:52:19,device3,ibeacon,6.747991,-1961096011,False,1631743077679,76,-81,-55,NaN,10001,62178,8a21c76b-74e3-4341-92be-33fcfdefdd63
1,2021-09-17 18:52:20,device3,ibeacon,3.705535,-1725979067,False,1631742537333,76,-70,-55,NaN,10001,1115,8a21c76b-74e3-4341-92be-33fcfdefdd63
2,2021-09-17 18:52:20,device3,ibeacon,15.189420,-1270774290,False,1631743079933,76,-90,-54,NaN,4096,8198,56db0365-a001-4062-9e4d-499d3b8eccf3
3,2021-09-17 21:22:02,device3,ibeacon,6.468056,-1270773610,False,1631743081044,76,-79,-54,NaN,4096,8206,56db0365-a001-4062-9e4d-499d3b8eccf3
4,2021-09-17 21:22:08,device3,eddystone_url,111.501470,297343086,False,1631901060285,65194,-86,-39,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15342,2021-11-01 01:16:44,device3,ibeacon,20.728643,-1060113490,False,1633905798697,76,-96,-55,NaN,1,0,2686f39c-bada-4658-854a-a62e7e5e8b8d
15343,2021-11-01 01:16:44,device3,ibeacon,19.312760,-1060113490,False,1633905800994,76,-94,-55,NaN,1,0,2686f39c-bada-4658-854a-a62e7e5e8b8d
15344,2021-11-01 01:16:44,device3,ibeacon,18.637318,-1060113490,False,1633905847804,76,-94,-55,NaN,1,0,2686f39c-bada-4658-854a-a62e7e5e8b8d
15345,2021-11-01 01:16:44,device3,ibeacon,16.139426,-1060113490,False,1633905872907,76,-94,-55,NaN,1,0,2686f39c-bada-4658-854a-a62e7e5e8b8d


In [22]:
filenames = ['20210924-210927','20211002-000000','20211007-192318','20211009-193250','20211009-202648',\
            '20211010-171904','20211010-183951','20211014-203426','20211014-211841','20211015-151732']

gps_device3 = []

for file in filenames:
    GPS = pd.read_csv('/Users/smritimohta/Desktop/Capstone/Final/Code/GPS Logger Collection/Device3/device3 - '+file+'.txt',header=0)
    gps_device3.append(GPS)
    
gps_df = pd.concat(gps_device3, axis=0, ignore_index=True)
gps_df['date time']= pd.to_datetime(gps_df['date time'])

In [23]:
gps_df

,type,date time,latitude,longitude,accuracy(m),altitude(m),geoid_height(m),speed(m/s),bearing(deg),sat_used,sat_inview,name,desc
0,T,2021-09-24 23:28:29,40.672338,-73.970412,30,21.0,NaN,0.0,NaN,5.0,22,20210924-192829,GPS Logger: 20210924-192829
1,T,2021-09-24 23:28:30,40.672340,-73.970417,26,22.0,NaN,0.0,NaN,5.0,22,NaN,NaN
2,T,2021-09-24 23:28:31,40.672341,-73.970418,23,22.0,NaN,0.0,NaN,5.0,22,NaN,NaN
3,T,2021-09-24 23:28:32,40.672341,-73.970419,23,22.0,NaN,0.0,NaN,5.0,22,NaN,NaN
4,T,2021-09-24 23:28:33,40.672341,-73.970420,23,22.0,NaN,0.0,NaN,5.0,22,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
20609,T,2021-10-15 19:34:19,40.752863,-74.001682,32,-4.0,NaN,0.0,NaN,11.0,20,NaN,NaN
20610,T,2021-10-15 19:34:20,40.752863,-74.001684,31,-4.0,NaN,0.0,NaN,11.0,20,NaN,NaN
20611,T,2021-10-15 19:34:21,40.752862,-74.001687,31,-4.0,NaN,0.0,NaN,11.0,20,NaN,NaN
20612,T,2021-10-15 19:34:22,40.752863,-74.001688,30,-4.0,NaN,0.0,NaN,15.0,20,NaN,NaN


In [25]:
df_device3 = finaldf.merge(gps_df, left_on='timestamp', right_on='date time')
df_device3

,timestamp,name_x,beacon_type,distance,hashcode,isBlocked,last_seen,manufacturer,rssi,tx_power,...,longitude,accuracy(m),altitude(m),geoid_height(m),speed(m/s),bearing(deg),sat_used,sat_inview,name_y,desc
0,2021-09-24 23:29:05,device3,ibeacon,8.200326,-771085463,False,1631914151482,76,-92,-59,...,-73.970296,6,12.0,NaN,0.00,NaN,9.0,22,NaN,NaN
1,2021-09-24 23:29:05,device3,ibeacon,8.200326,-771085463,False,1631914151482,76,-92,-59,...,-73.970296,6,12.0,NaN,0.00,NaN,9.0,22,NaN,NaN
2,2021-09-24 23:29:05,device3,ibeacon,8.200326,-771085463,False,1631914151482,76,-92,-59,...,-73.970296,6,12.0,NaN,0.00,NaN,9.0,22,NaN,NaN
3,2021-09-24 23:29:05,device3,ibeacon,8.350944,-771085463,False,1631914152611,76,-94,-59,...,-73.970296,6,12.0,NaN,0.00,NaN,9.0,22,NaN,NaN
4,2021-09-24 23:29:05,device3,ibeacon,8.131318,-771085463,False,1631914154844,76,-86,-59,...,-73.970296,6,12.0,NaN,0.00,NaN,9.0,22,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12068,2021-10-15 19:33:17,device3,ibeacon,2.584403,-1961984076,False,1633826476335,76,-69,-55,...,-74.001548,19,-6.0,NaN,0.58,281.0,7.0,22,NaN,NaN
12069,2021-10-15 19:34:12,device3,ibeacon,7.299538,-1725973147,False,1633826360022,76,-81,-55,...,-74.001669,31,-5.0,NaN,0.00,NaN,11.0,20,NaN,NaN
12070,2021-10-15 19:34:14,device3,eddystone_uid,4.255900,-1440638870,False,1633826155115,65194,-93,-68,...,-74.001674,32,-4.0,NaN,0.00,NaN,10.0,20,NaN,NaN
12071,2021-10-15 19:34:14,device3,ibeacon,17.604589,-1270767718,False,1633826253664,76,-92,-54,...,-74.001674,32,-4.0,NaN,0.00,NaN,10.0,20,NaN,NaN


## device 4

In [32]:
records = []

file_path = '/Users/smritimohta/Desktop/Capstone/Final/Code/device4.json'
with open(file_path) as f:
    for line in f:
        records.append(json.loads(line))
main = pd.DataFrame.from_dict(records)
beacons = main["beacons"]
beacondetails = pd.DataFrame(list(chain.from_iterable(beacons)))
beacondata = beacondetails["ibeacon_data"]
beacondatadf = pd.DataFrame(beacondata)
beacondatadetails = beacondatadf["ibeacon_data"].apply(pd.Series)
detailsdf = pd.concat([main, beacondetails, beacondatadetails],axis=1)
cleandf = detailsdf[['timestamp','name','beacon_type','distance','hashcode','isBlocked','last_seen','manufacturer','rssi','tx_power','eddystoneUidData','major','minor','uuid']]
finaldf = cleandf[cleandf["timestamp"].notnull()]
finaldf['timestamp']=pd.to_datetime(finaldf['timestamp'])

<ipython-input-32-6e80f0368ec5>:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  finaldf['timestamp']=pd.to_datetime(finaldf['timestamp'])


In [33]:
finaldf

,timestamp,name,beacon_type,distance,hashcode,isBlocked,last_seen,manufacturer,rssi,tx_power,eddystoneUidData,major,minor,uuid
0,2021-09-17 18:07:23,device4,ibeacon,0.053491,-754952808,False,1631902042835,76,-49,-65,NaN,123,1,80b4e91d-e399-44ba-ac61-fbf524b59f5a
1,2021-09-17 18:07:23,device4,ibeacon,0.053491,-754952808,False,1631902042835,76,-49,-65,NaN,123,1,80b4e91d-e399-44ba-ac61-fbf524b59f5a
2,2021-09-17 18:07:35,device4,ibeacon,0.073396,-754952808,False,1631902055119,76,-50,-65,NaN,123,1,80b4e91d-e399-44ba-ac61-fbf524b59f5a
3,2021-09-17 18:07:43,device4,ibeacon,0.074230,-754952808,False,1631902062968,76,-51,-65,NaN,123,1,80b4e91d-e399-44ba-ac61-fbf524b59f5a
4,2021-09-28 19:40:46,device4,eddystone_uid,5.031166,-2093960133,False,1632259747141,65194,-85,-60,"{'instance_id': '0x0c00255c0000', 'namespace_i...",NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12835,2021-11-17 19:12:39,device4,ibeacon,10.046016,-1213521970,False,1636055783999,76,-90,-50,NaN,27884,35600,50765cb7-d9ea-4e21-99a4-fa879613a492
12836,2021-11-17 19:12:39,device4,ibeacon,12.016902,-1060113490,False,1636055789600,76,-91,-55,NaN,1,0,2686f39c-bada-4658-854a-a62e7e5e8b8d
12837,2021-11-17 19:12:39,device4,ibeacon,11.574292,-1060113490,False,1636055790717,76,-88,-55,NaN,1,0,2686f39c-bada-4658-854a-a62e7e5e8b8d
12838,2021-11-17 19:12:44,device4,ibeacon,11.574292,-1060113490,False,1636055790717,76,-88,-55,NaN,1,0,2686f39c-bada-4658-854a-a62e7e5e8b8d


In [34]:
filenames = ['20210915-174554','20210921-172831','20210928-154234','20211009-150931','20211012-180812',\
            '20211013-215252','20211014-185509','20211014-231325','20211016-130450','20211016-164835',\
            '20211017-141852','20211106-152928','20211117-115031','20211117-121221','20211117-135931']

gps_device4 = []

for file in filenames:
    GPS = pd.read_csv('/Users/smritimohta/Desktop/Capstone/Final/Code/GPS Logger Collection/Device4/device4 - '+file+'.txt',header=0)
    gps_device4.append(GPS)
    
gps_df = pd.concat(gps_device4, axis=0, ignore_index=True)
gps_df['date time']= pd.to_datetime(gps_df['date time'])

In [35]:
gps_df

,type,date time,latitude,longitude,accuracy(m),altitude(m),geoid_height(m),speed(m/s),bearing(deg),sat_used,sat_inview,name,desc
0,T,2021-09-15 21:45:54,40.717478,-73.999105,20,-8.0,NaN,0.74,279.0,10.0,22,20210915-174554,GPS Logger: 20210915-174554
1,T,2021-09-15 21:45:55,40.717446,-73.999134,17,-8.0,NaN,1.18,288.0,10.0,22,NaN,NaN
2,T,2021-09-15 21:45:56,40.717450,-73.999144,14,-9.0,NaN,1.39,295.0,10.0,22,NaN,NaN
3,T,2021-09-15 21:45:57,40.717463,-73.999178,10,-11.0,NaN,2.04,318.0,9.0,22,NaN,NaN
4,T,2021-09-15 21:45:58,40.717456,-73.999230,10,-14.0,NaN,2.09,318.0,9.0,22,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
24376,T,2021-11-17 19:12:44,40.733093,-74.000233,6,-13.0,NaN,1.03,296.0,12.0,22,NaN,NaN
24377,T,2021-11-17 19:12:45,40.733090,-74.000234,7,-13.0,NaN,0.00,NaN,10.0,22,NaN,NaN
24378,T,2021-11-17 19:12:46,40.733087,-74.000222,7,-13.0,NaN,0.70,296.0,10.0,22,NaN,NaN
24379,T,2021-11-17 19:12:47,40.733084,-74.000222,6,-13.0,NaN,0.41,296.0,10.0,22,NaN,NaN


In [36]:
df_device4 = finaldf.merge(gps_df, left_on='timestamp', right_on='date time')
df_device4

,timestamp,name_x,beacon_type,distance,hashcode,isBlocked,last_seen,manufacturer,rssi,tx_power,...,longitude,accuracy(m),altitude(m),geoid_height(m),speed(m/s),bearing(deg),sat_used,sat_inview,name_y,desc
0,2021-09-28 19:42:44,device4,eddystone_uid,5.683823,-52086734,False,1632259746013,65194,-86,-60,...,-73.980075,27,44.0,NaN,2.79,215.0,5.0,22,NaN,NaN
1,2021-09-28 19:42:44,device4,ibeacon,14.787390,-47281815,False,1632259861874,76,-81,-50,...,-73.980075,27,44.0,NaN,2.79,215.0,5.0,22,NaN,NaN
2,2021-09-28 19:42:44,device4,eddystone_uid,4.561479,90464375,False,1632259745955,65194,-80,-60,...,-73.980075,27,44.0,NaN,2.79,215.0,5.0,22,NaN,NaN
3,2021-09-28 19:42:53,device4,ibeacon,3.349874,145574607,False,1632259943631,76,-76,-59,...,-73.979918,45,22.0,NaN,NaN,NaN,3.0,22,NaN,NaN
4,2021-09-28 19:42:54,device4,ibeacon,2.645353,147513809,False,1632260298575,76,-82,-59,...,-73.979942,52,23.0,NaN,NaN,NaN,4.0,22,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7295,2021-11-17 19:12:39,device4,ibeacon,10.046016,-1213521970,False,1636055783999,76,-90,-50,...,-74.000181,4,-11.0,NaN,1.15,297.0,11.0,22,NaN,NaN
7296,2021-11-17 19:12:39,device4,ibeacon,12.016902,-1060113490,False,1636055789600,76,-91,-55,...,-74.000181,4,-11.0,NaN,1.15,297.0,11.0,22,NaN,NaN
7297,2021-11-17 19:12:39,device4,ibeacon,11.574292,-1060113490,False,1636055790717,76,-88,-55,...,-74.000181,4,-11.0,NaN,1.15,297.0,11.0,22,NaN,NaN
7298,2021-11-17 19:12:44,device4,ibeacon,11.574292,-1060113490,False,1636055790717,76,-88,-55,...,-74.000233,6,-13.0,NaN,1.03,296.0,12.0,22,NaN,NaN


## smriti phone

In [26]:
records = []

file_path = '/Users/smritimohta/Desktop/Capstone/Final/Code/smritiphone.json'
with open(file_path) as f:
    for line in f:
        records.append(json.loads(line))
main = pd.DataFrame.from_dict(records)
beacons = main["beacons"]
beacondetails = pd.DataFrame(list(chain.from_iterable(beacons)))
beacondata = beacondetails["ibeacon_data"]
beacondatadf = pd.DataFrame(beacondata)
beacondatadetails = beacondatadf["ibeacon_data"].apply(pd.Series)
detailsdf = pd.concat([main, beacondetails, beacondatadetails],axis=1)
cleandf = detailsdf[['timestamp','name','beacon_type','distance','hashcode','isBlocked','last_seen','manufacturer','rssi','tx_power','eddystoneUidData','major','minor','uuid']]
finaldf = cleandf[cleandf["timestamp"].notnull()]
finaldf['timestamp']=pd.to_datetime(finaldf['timestamp'])

<ipython-input-26-0952fd011373>:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  finaldf['timestamp']=pd.to_datetime(finaldf['timestamp'])


In [27]:
finaldf

,timestamp,name,beacon_type,distance,hashcode,isBlocked,last_seen,manufacturer,rssi,tx_power,eddystoneUidData,major,minor,uuid
0,2021-08-06 01:22:53,smriti phone,ibeacon,2.488987,-1109389005,False,1628212971034,76,-76,-61,NaN,14364,38134,5993a94c-7d97-4df7-9abf-e493bfd5d000
1,2021-08-06 01:22:53,smriti phone,ibeacon,2.488987,-1109389005,False,1628212971034,76,-76,-61,NaN,14364,38134,5993a94c-7d97-4df7-9abf-e493bfd5d000
2,2021-08-06 01:22:54,smriti phone,ibeacon,4.436567,673625894,False,1628212972136,76,-84,-61,NaN,51208,8356,5993a94c-7d97-4df7-9abf-e493bfd5d000
3,2021-08-06 01:22:56,smriti phone,ibeacon,4.600171,-1646189899,False,1628212973252,76,-83,-61,NaN,23247,52585,5993a94c-7d97-4df7-9abf-e493bfd5d000
4,2021-08-06 01:22:56,smriti phone,ibeacon,3.319158,-1109389005,False,1628212973246,76,-83,-61,NaN,14364,38134,5993a94c-7d97-4df7-9abf-e493bfd5d000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7146,2021-09-14 18:36:56,smriti phone,ibeacon,7.327713,-2063510500,False,1628298374350,76,-91,-61,NaN,2913,15911,5993a94c-7d97-4df7-9abf-e493bfd5d000
7147,2021-09-14 18:36:57,smriti phone,ibeacon,5.919498,-1250255759,False,1628298375395,76,-87,-61,NaN,44965,39946,5993a94c-7d97-4df7-9abf-e493bfd5d000
7148,2021-09-14 18:36:57,smriti phone,ibeacon,4.544994,-1191513573,False,1628298374364,76,-83,-61,NaN,23983,8697,5993a94c-7d97-4df7-9abf-e493bfd5d000
7149,2021-09-14 18:36:57,smriti phone,ibeacon,5.065573,-1109389005,False,1628298374331,76,-83,-61,NaN,14364,38134,5993a94c-7d97-4df7-9abf-e493bfd5d000


In [28]:
filenames = ['20210908-184751','20210914-142226']

gps_smritiphone = []

for file in filenames:
    GPS = pd.read_csv('/Users/smritimohta/Desktop/Capstone/Final/Code/GPS Logger Collection/SmritiPhone/device1 - '+file+'.txt',header=0)
    gps_smritiphone.append(GPS)
    
gps_df = pd.concat(gps_smritiphone, axis=0, ignore_index=True)
gps_df['date time']= pd.to_datetime(gps_df['date time'])

In [29]:
gps_df

,type,date time,latitude,longitude,accuracy(m),altitude(m),geoid_height(m),speed(m/s),bearing(deg),sat_used,sat_inview,name,desc
0,T,2021-09-08 22:47:51,40.692926,-73.986387,19,28.938,NaN,0.00,NaN,21.0,26,20210908-184751,GPS Logger: 20210908-184751
1,T,2021-09-08 22:47:52,40.692930,-73.986386,18,26.942,NaN,0.00,NaN,21.0,26,NaN,NaN
2,T,2021-09-08 22:47:54,40.692903,-73.986363,17,25.957,NaN,0.44,338.0,16.0,26,NaN,NaN
3,T,2021-09-08 22:47:55,40.692900,-73.986359,17,25.997,NaN,0.65,331.0,15.0,26,NaN,NaN
4,T,2021-09-08 22:47:56,40.692875,-73.986331,17,25.700,NaN,0.48,352.0,17.0,26,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
905,T,2021-09-14 18:37:00,40.694427,-73.985985,5,-15.519,NaN,0.91,283.0,19.0,23,NaN,NaN
906,T,2021-09-14 18:37:01,40.694427,-73.985999,5,-15.818,NaN,0.74,270.0,20.0,23,NaN,NaN
907,T,2021-09-14 18:37:02,40.694424,-73.986013,4,-16.178,NaN,0.73,265.0,20.0,23,NaN,NaN
908,T,2021-09-14 18:37:03,40.694425,-73.986027,5,-16.759,NaN,0.75,283.0,20.0,23,NaN,NaN


In [30]:
df_smritiphone = finaldf.merge(gps_df, left_on='timestamp', right_on='date time')
df_smritiphone

,timestamp,name_x,beacon_type,distance,hashcode,isBlocked,last_seen,manufacturer,rssi,tx_power,...,longitude,accuracy(m),altitude(m),geoid_height(m),speed(m/s),bearing(deg),sat_used,sat_inview,name_y,desc
0,2021-09-14 18:22:45,smriti phone,ibeacon,50.990363,-1060113490,False,1628298167545,76,-94,-47,...,-73.986798,9,63.187,NaN,1.02,141.0,18.0,26,NaN,NaN
1,2021-09-14 18:22:48,smriti phone,ibeacon,50.990363,-1060113490,False,1628298167545,76,-94,-47,...,-73.986724,7,55.072,NaN,1.24,109.0,18.0,26,NaN,NaN
2,2021-09-14 18:22:50,smriti phone,ibeacon,50.990363,-1060113490,False,1628298167545,76,-94,-47,...,-73.986652,6,49.960,NaN,1.15,122.0,19.0,26,NaN,NaN
3,2021-09-14 18:22:52,smriti phone,ibeacon,50.448288,-1060113490,False,1628298168647,76,-92,-47,...,-73.986589,6,48.363,NaN,1.20,116.0,19.0,26,NaN,NaN
4,2021-09-14 18:22:53,smriti phone,ibeacon,50.448288,-1060113490,False,1628298168646,76,-92,-47,...,-73.986564,5,47.336,NaN,1.07,114.0,19.0,26,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
523,2021-09-14 18:36:56,smriti phone,ibeacon,7.327713,-2063510500,False,1628298374350,76,-91,-61,...,-73.985955,4,-13.745,NaN,0.86,313.0,19.0,23,NaN,NaN
524,2021-09-14 18:36:57,smriti phone,ibeacon,5.919498,-1250255759,False,1628298375395,76,-87,-61,...,-73.985962,4,-14.355,NaN,0.78,329.0,18.0,23,NaN,NaN
525,2021-09-14 18:36:57,smriti phone,ibeacon,4.544994,-1191513573,False,1628298374364,76,-83,-61,...,-73.985962,4,-14.355,NaN,0.78,329.0,18.0,23,NaN,NaN
526,2021-09-14 18:36:57,smriti phone,ibeacon,5.065573,-1109389005,False,1628298374331,76,-83,-61,...,-73.985962,4,-14.355,NaN,0.78,329.0,18.0,23,NaN,NaN


## merge all dataframes

In [37]:
dataframes = [df_device1,df_device3,df_device4,df_smritiphone]


In [38]:
df = pd.concat(dataframes,ignore_index=True)
df

,timestamp,name_x,beacon_type,distance,hashcode,isBlocked,last_seen,manufacturer,rssi,tx_power,...,longitude,accuracy(m),altitude(m),geoid_height(m),speed(m/s),bearing(deg),sat_used,sat_inview,name_y,desc
0,2021-09-24 19:18:47,device1,ibeacon,4.600171,-1229677222,False,1632351432738,76,-81,-61,...,-74.012588,5,-33.543,NaN,0.87,360.0,20.0,31,NaN,NaN
1,2021-09-24 19:18:48,device1,ibeacon,6.358023,32202963,False,1632351432737,76,-88,-61,...,-74.012594,5,-33.221,NaN,0.73,359.0,20.0,31,NaN,NaN
2,2021-09-24 19:19:16,device1,ibeacon,4.040893,144680941,False,1632511100375,76,-80,-59,...,-74.012693,6,-32.870,NaN,0.00,NaN,21.0,31,NaN,NaN
3,2021-09-24 19:19:23,device1,ibeacon,8.462784,144680941,False,1632511125686,76,-90,-59,...,-74.012701,6,-32.694,NaN,0.00,NaN,21.0,31,NaN,NaN
4,2021-09-24 19:19:25,device1,ibeacon,7.056213,144680941,False,1632511127891,76,-85,-59,...,-74.012702,6,-32.653,NaN,0.00,NaN,21.0,31,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
346882,2021-09-14 18:36:56,smriti phone,ibeacon,7.327713,-2063510500,False,1628298374350,76,-91,-61,...,-73.985955,4,-13.745,NaN,0.86,313.0,19.0,23,NaN,NaN
346883,2021-09-14 18:36:57,smriti phone,ibeacon,5.919498,-1250255759,False,1628298375395,76,-87,-61,...,-73.985962,4,-14.355,NaN,0.78,329.0,18.0,23,NaN,NaN
346884,2021-09-14 18:36:57,smriti phone,ibeacon,4.544994,-1191513573,False,1628298374364,76,-83,-61,...,-73.985962,4,-14.355,NaN,0.78,329.0,18.0,23,NaN,NaN
346885,2021-09-14 18:36:57,smriti phone,ibeacon,5.065573,-1109389005,False,1628298374331,76,-83,-61,...,-73.985962,4,-14.355,NaN,0.78,329.0,18.0,23,NaN,NaN


In [39]:
df.columns

Index(['timestamp', 'name_x', 'beacon_type', 'distance', 'hashcode',
       'isBlocked', 'last_seen', 'manufacturer', 'rssi', 'tx_power',
       'eddystoneUidData', 'major', 'minor', 'uuid', 'type', 'date time',
       'latitude', 'longitude', 'accuracy(m)', 'altitude(m)',
       'geoid_height(m)', 'speed(m/s)', 'bearing(deg)', 'sat_used',
       'sat_inview', 'name_y', 'desc'],
      dtype='object')

In [40]:
df.to_csv('/Users/smritimohta/Desktop/Capstone/Final/Code/data.csv')